# Starbucks Capstone Challenge

### Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline

### Reading the json files

In [2]:
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

### Exploring json files and analyzing the features.

#### portfolio.json

In [3]:
print("Portfolio Size = {}".format(portfolio.shape))
portfolio.head()

Portfolio Size = (10, 6)


,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


#### profile.json file

In [4]:
print("Profile Size = {}".format(profile.shape))
profile.head()

Profile Size = (17000, 5)


,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


#### transcript.json

In [5]:
print("Transcript Size = {}".format(transcript.shape))
transcript.head()

Transcript Size = (306534, 4)


,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


### Checking for Nan values

In [6]:
def check_nan(dataframe):
    nan_columns = dataframe.columns[dataframe.isna().any()].tolist()
    total = len(dataframe)
    print("Total : {}".format(total))
    for column in nan_columns:
        count = dataframe[column].isna().sum()
        print("{} : {}, remaining : {}".format(column, count, total - count))

In [7]:
check_nan(portfolio)

Total : 10


In [8]:
portfolio_nan = portfolio

In [9]:
check_nan(profile)

Total : 17000
gender : 2175, remaining : 14825
income : 2175, remaining : 14825


In [10]:
profile_nan = profile.dropna()
profile_nan = profile_nan.reset_index(drop=True)
profile_nan

,gender,age,id,became_member_on,income
0,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
1,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
2,M,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0
3,M,65,389bc3fa690240e798340f5a15918d5c,20180209,53000.0
4,M,58,2eeac8d8feae4a8cad5a6af0499a211d,20171111,51000.0
...,...,...,...,...,...
14820,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0
14821,M,61,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0
14822,M,49,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0
14823,F,83,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0


In [11]:
check_nan(transcript)

Total : 306534


In [12]:
transcript_nan = transcript

### One-hot encoding

In [13]:
def do_one_hot_encoding(dataframe, column_list):
    for column in column_list:
        column_dummies = pd.get_dummies(dataframe[column])
        dataframe = dataframe.drop(column, axis = 1)
        dataframe = dataframe.join(column_dummies)
    return dataframe

In [14]:
columns = ['offer_type']
portfolio_encoded = do_one_hot_encoding(portfolio_nan, columns)
portfolio_encoded.head()

,reward,channels,difficulty,duration,id,bogo,discount,informational
0,10,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0
1,10,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0
2,0,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,0,0,1
3,5,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0
4,5,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0


In [15]:
columns = ['gender']
profile_encoded = do_one_hot_encoding(profile_nan, columns)
profile_encoded.head()

,age,id,became_member_on,income,F,M,O
0,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0,1,0,0
1,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,1,0,0
2,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0,0,1,0
3,65,389bc3fa690240e798340f5a15918d5c,20180209,53000.0,0,1,0
4,58,2eeac8d8feae4a8cad5a6af0499a211d,20171111,51000.0,0,1,0


In [16]:
columns = ['event']
transcript_encoded = do_one_hot_encoding(transcript_nan, columns)
transcript_encoded.head()

,person,value,time,offer completed,offer received,offer viewed,transaction
0,78afa995795e4d85b5d9ceeca43f5fef,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,1,0,0
1,a03223e636434f42ac4c3df47e8bac43,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0,1,0,0
2,e2127556f4f64592b11af22de27a7932,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,0,1,0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0,1,0,0
4,68617ca6246f4fbc85e91a2a49552598,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,1,0,0


In [17]:
all_keys  = []
def display_all_keys(x):
   
    if x.keys() not in all_keys:
        all_keys.append(x.keys())
        

In [18]:
transcript_encoded['value'].apply(lambda x: display_all_keys(x))
print(all_keys)

[dict_keys(['offer id']), dict_keys(['amount']), dict_keys(['offer_id', 'reward'])]


In [19]:
def change_name(x, from_name, to_name):
    if from_name in x.keys():
        x[to_name] = x.pop(from_name)
    return x

In [20]:
transcript_encoded['value'].apply(lambda x: change_name(x, 'offer id', 'offer_id'))

0         {'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1         {'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2         {'offer_id': '2906b810c7d4411798c6938adc9daaa5'}
3         {'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4         {'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
                                ...                       
306529                      {'amount': 1.5899999999999999}
306530                                    {'amount': 9.53}
306531                                    {'amount': 3.61}
306532                      {'amount': 3.5300000000000002}
306533                                    {'amount': 4.05}
Name: value, Length: 306534, dtype: object

In [21]:
all_keys = []
transcript_encoded['value'].apply(lambda x: display_all_keys(x))
print(all_keys)

[dict_keys(['offer_id']), dict_keys(['amount']), dict_keys(['offer_id', 'reward'])]


In [22]:
def check_not_offer(x, column):
    if column in x.keys() :
        return x[column]

In [23]:
offer_ids = transcript_encoded['value'].apply(lambda x: check_not_offer(x, 'offer_id'))

In [24]:
offer_ids = offer_ids.to_frame()
offer_ids.columns = ['offer_id']
offer_ids.head()

,offer_id
0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,2906b810c7d4411798c6938adc9daaa5
3,fafdcd668e3743c1bb461111dcafc2a4
4,4d5c57ea9a6940dd891ad53e9dbe8da0


In [25]:
transcript_offer_id = transcript_encoded.join(offer_ids)

In [26]:
transcript_offer_id

,person,value,time,offer completed,offer received,offer viewed,transaction,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,{'offer_id': '2906b810c7d4411798c6938adc9daaa5'},0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0
...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,{'amount': 1.5899999999999999},714,0,0,0,1,None
306530,68213b08d99a4ae1b0dcb72aebd9aa35,{'amount': 9.53},714,0,0,0,1,None
306531,a00058cf10334a308c68e7631c529907,{'amount': 3.61},714,0,0,0,1,None
306532,76ddbd6576844afe811f1a3c0fbb5bec,{'amount': 3.5300000000000002},714,0,0,0,1,None


In [27]:
column = 'amount'
offer_ids = transcript_encoded['value'].apply(lambda x: check_not_offer(x, column))
offer_ids = offer_ids.to_frame()
offer_ids.columns = [column]
offer_ids

,amount
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
306529,1.59
306530,9.53
306531,3.61
306532,3.53


In [28]:
transcript_amount = transcript_offer_id.join(offer_ids)
transcript_amount.head()

,person,value,time,offer completed,offer received,offer viewed,transaction,offer_id,amount
0,78afa995795e4d85b5d9ceeca43f5fef,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
1,a03223e636434f42ac4c3df47e8bac43,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
2,e2127556f4f64592b11af22de27a7932,{'offer_id': '2906b810c7d4411798c6938adc9daaa5'},0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN
4,68617ca6246f4fbc85e91a2a49552598,{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN


In [29]:
column = 'reward'
offer_ids = transcript_encoded['value'].apply(lambda x: check_not_offer(x, column))
offer_ids = offer_ids.to_frame()
offer_ids.columns = [column]
offer_ids.dropna()

,reward
12658,2.0
12672,2.0
12679,5.0
12692,10.0
12697,10.0
...,...
306475,3.0
306497,3.0
306506,2.0
306509,2.0


In [30]:
transcript_reward = transcript_amount.join(offer_ids)
transcript_reward.head()

,person,value,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,{'offer_id': '2906b810c7d4411798c6938adc9daaa5'},0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,{'offer_id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN


In [31]:
transcript_on_value = transcript_reward.drop('value', axis = 1)
transcript_on_value

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,714,0,0,0,1,None,1.59,NaN
306530,68213b08d99a4ae1b0dcb72aebd9aa35,714,0,0,0,1,None,9.53,NaN
306531,a00058cf10334a308c68e7631c529907,714,0,0,0,1,None,3.61,NaN
306532,76ddbd6576844afe811f1a3c0fbb5bec,714,0,0,0,1,None,3.53,NaN


In [32]:
portfolio_encoded

,reward,channels,difficulty,duration,id,bogo,discount,informational
0,10,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0
1,10,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0
2,0,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,0,0,1
3,5,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0
4,5,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0
5,3,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0
6,2,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0
7,0,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1
8,5,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0
9,2,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0


In [33]:
def list_to_dataframe(x, column):
    if column in x:
        return 1
    else:
        return 0

In [34]:
values = ['email', 'web', 'mobile', 'social']
for i in values:
    portfolio_encoded_series = portfolio_encoded['channels'].apply(lambda x: list_to_dataframe(x, i))
    portfolio_encoded_series = portfolio_encoded_series.to_frame()
    portfolio_encoded_series.columns = [i]
    portfolio_encoded = portfolio_encoded.join(portfolio_encoded_series)
portfolio_encoded

,reward,channels,difficulty,duration,id,bogo,discount,informational,email,web,mobile,social
0,10,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,1,0,1,1
1,10,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1,1
2,0,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,1,0
3,5,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,1,0
4,5,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,1,0,0
5,3,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1,1
6,2,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1,1
7,0,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,1,0,1,1
8,5,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1,1
9,2,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,1,0


In [35]:
portfolio_channels = portfolio_encoded.drop('channels', axis = 1)
portfolio_channels

,reward,difficulty,duration,id,bogo,discount,informational,email,web,mobile,social
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,1,0,1,1
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1,1
2,0,0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,1,0
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,1,0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,1,0,0
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1,1
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1,1
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,1,0,1,1
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1,1
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,1,0


In [36]:
#if 'email' is same in every row then its redundant
portfolio_channels = portfolio_channels.drop('email', axis = 1)
portfolio_channels

,reward,difficulty,duration,id,bogo,discount,informational,web,mobile,social
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,0,1,1
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1
2,0,0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,0
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,0,0
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,0,1,1
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,0


### After encoding

In [37]:
portfolio_channels

,reward,difficulty,duration,id,bogo,discount,informational,web,mobile,social
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,0,1,1
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1
2,0,0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,0
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,0,0
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,0,1,1
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,0


In [38]:
transcript_on_value

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,714,0,0,0,1,None,1.59,NaN
306530,68213b08d99a4ae1b0dcb72aebd9aa35,714,0,0,0,1,None,9.53,NaN
306531,a00058cf10334a308c68e7631c529907,714,0,0,0,1,None,3.61,NaN
306532,76ddbd6576844afe811f1a3c0fbb5bec,714,0,0,0,1,None,3.53,NaN


In [39]:
profile_encoded

,age,id,became_member_on,income,F,M,O
0,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0,1,0,0
1,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0,1,0,0
2,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0,0,1,0
3,65,389bc3fa690240e798340f5a15918d5c,20180209,53000.0,0,1,0
4,58,2eeac8d8feae4a8cad5a6af0499a211d,20171111,51000.0,0,1,0
...,...,...,...,...,...,...,...
14820,45,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,54000.0,1,0,0
14821,61,2cb4f97358b841b9a9773a7aa05a9d77,20180713,72000.0,0,1,0
14822,49,01d26f638c274aa0b965d24cefe3183f,20170126,73000.0,0,1,0
14823,83,9dc1421481194dcd9400aec7c9ae6366,20160307,50000.0,1,0,0


In [40]:
transcript_on_value['reward'].fillna(0, inplace=True)
transcript_on_value

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,0.0
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,0.0
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,NaN,0.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,0.0
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,0.0
...,...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,714,0,0,0,1,None,1.59,0.0
306530,68213b08d99a4ae1b0dcb72aebd9aa35,714,0,0,0,1,None,9.53,0.0
306531,a00058cf10334a308c68e7631c529907,714,0,0,0,1,None,3.61,0.0
306532,76ddbd6576844afe811f1a3c0fbb5bec,714,0,0,0,1,None,3.53,0.0


In [41]:
transcript_on_value['amount'].fillna(0, inplace=True)
transcript_on_value

,person,time,offer completed,offer received,offer viewed,transaction,offer_id,amount,reward
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.00,0.0
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.00,0.0
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,0.00,0.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,0.00,0.0
4,68617ca6246f4fbc85e91a2a49552598,0,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0.00,0.0
...,...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,714,0,0,0,1,None,1.59,0.0
306530,68213b08d99a4ae1b0dcb72aebd9aa35,714,0,0,0,1,None,9.53,0.0
306531,a00058cf10334a308c68e7631c529907,714,0,0,0,1,None,3.61,0.0
306532,76ddbd6576844afe811f1a3c0fbb5bec,714,0,0,0,1,None,3.53,0.0


In [42]:
p_columns = portfolio_channels.columns
p_columns = p_columns.to_list()

In [43]:
for n, i in enumerate(p_columns):
    if i!='offer_id':
        p_columns[n] = 'offer_' + p_columns[n]
p_columns

['offer_reward',
 'offer_difficulty',
 'offer_duration',
 'offer_id',
 'offer_bogo',
 'offer_discount',
 'offer_informational',
 'offer_web',
 'offer_mobile',
 'offer_social']

In [44]:
portfolio_channels.columns = p_columns
portfolio_channels

,offer_reward,offer_difficulty,offer_duration,offer_id,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,0,1,1
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1
2,0,0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,0
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,0,0
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,0,1,1
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,0


In [51]:
portfolio_channels.to_csv("portfolio_formatted.csv", index=False)

In [52]:
transcript_on_value.to_csv("transcript_formatted.csv", index=False)

In [53]:
profile_encoded.to_csv("profile_formatted.csv", index=False)

In [54]:
portfolio_new = pd.read_csv("portfolio_formatted.csv")
transcript_new = pd.read_csv("transcript_formatted.csv")
profile_new = pd.read_csv("profile_formatted.csv")

In [55]:
portfolio_new

,offer_reward,offer_difficulty,offer_duration,offer_id,offer_bogo,offer_discount,offer_informational,offer_web,offer_mobile,offer_social
0,10,10,7,ae264e3637204a6fb9bb56bc8210ddfd,1,0,0,0,1,1
1,10,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,1,0,0,1,1,1
2,0,0,4,3f207df678b143eea3cee63160fa8bed,0,0,1,1,1,0
3,5,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,0,0,1,1,0
4,5,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,1,0,1,0,0
5,3,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,0,1,0,1,1,1
6,2,10,10,fafdcd668e3743c1bb461111dcafc2a4,0,1,0,1,1,1
7,0,0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,1,0,1,1
8,5,5,5,f19421c1d4aa40978ebb69ca19b0e20d,1,0,0,1,1,1
9,2,10,7,2906b810c7d4411798c6938adc9daaa5,0,1,0,1,1,0
